In [1]:
#install needed packages for Diffbot
!pip install requests

In [101]:
############################
##### UPDATE AS NEEDED #####
############################
# search articles containing the following tags
entities=["Natalie Portman","Serena Williams","Zac Efron","Benedict Cumberbatch"]


# search articles written within the last X days
x_days="60"

# developer token id
token="d6c57dd8e326fce7aad8a11e297efe00"

In [102]:
# Search with DQL via python API
# store tag results in list
import urllib.parse
import requests
import json
import pandas as pd

headers = {"Accept": "application/json"}
results=list()

for entity in entities:
    encoded_entity=urllib.parse.quote(entity)
    ####!!!! Max number of results returned is 1000
    url ="https://kg.diffbot.com/kg/v3/dql?token="+token+"&type=query&query=type%3AArticle%20date%3C"+x_days+"d%20tags.label%3A%22"+encoded_entity+"%22%20facet%3Atags.label&col=all&from=0&size=1000&format=json&jsonmode=%20&nonCanonicalFacts=false&noDedupArticles=false&cluster=false"
    response = requests.get(url, headers=headers)
    d = json.loads(response.text)
    df=pd.json_normalize(d,"data")
    results.append(df)


In [103]:
# preview results
len(results[1])

1000

In [104]:
results[1][["count","value"]].head()

,count,value
0,6368,Serena Williams
1,3406,Wimbledon Championships
2,2280,Grand Slam of tennis
3,1309,French Open
4,1184,Venus Williams


In [105]:
# Join dataframes to find common tags
similarities=()

for i in range(1,len(entities)):
    suf='_'+str(i)
    if i==1:
        join=results[0][["count","value"]].merge(results[i][["count","value"]], how='inner', on='value', suffixes=('_0', suf))
        similarities=join.copy()
    if i>1:
        join=similarities.merge(results[i][["count","value"]], how='inner', on='value', suffixes=(suf))
        similarities=join


/var/folders/t1/njfw6dkd30j6q9tb_njj8vn40000gn/T/ipykernel_21079/1483902753.py:10: FutureWarning: Passing 'suffixes' as a <class 'str'>, is not supported and may give unexpected results. Provide 'suffixes' as a tuple instead. In the future a 'TypeError' will be raised.
  join=similarities.merge(results[i][["count","value"]], how='inner', on='value', suffixes=(suf))
/var/folders/t1/njfw6dkd30j6q9tb_njj8vn40000gn/T/ipykernel_21079/1483902753.py:10: FutureWarning: Passing 'suffixes' as a <class 'str'>, is not supported and may give unexpected results. Provide 'suffixes' as a tuple instead. In the future a 'TypeError' will be raised.
  join=similarities.merge(results[i][["count","value"]], how='inner', on='value', suffixes=(suf))


In [106]:
similarities

,count_0,value,count_1,count_,count3
0,2739,Natalie Portman,31,1,9
1,1661,Chris Hemsworth,4,4,20
2,135,Tom Cruise,11,7,113
3,82,United States of America,678,4,35
4,71,Hollywood,12,34,12
5,67,India,23,2,9
6,58,Los Angeles,36,8,32
7,58,Los Angeles,36,1,32
8,37,Canada,54,1,24
9,36,Academy Awards,21,3,10


In [148]:
for i in range(len(entities)): print(sum(results[0].value.str.contains("2022 FIFA World Cup")))

1
1
1
1
